## Notes
* numpy doc: [Overview — NumPy Manual](https://docs.scipy.org/doc/numpy/)
* matplotlib.pyplot doc: [Matplotlib: Python plotting](https://matplotlib.org/)
* what is seaborn (TODO)

In [ ]:
import numpy as np
import matplotlib.pyplot as pp
import seaborn
import urllib.request as ureq
import sys
print('numpy version: {}'.format(np.__version__))
print('seaborn version: {}'.format(seaborn.__version__))
print('matplotlib version: {}'.format(sys.modules[pp.__package__].__version__))
print('urllib version: {}'.format(ureq.__version__))

In [ ]:
%matplotlib inline

## Get Data
```
ureq.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt','ghcnd-readme.txt')
ureq.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt','stations.txt')
# {'USW00014922': 'MN MINNEAPOLIS/ST PAUL AP GSN HCN 72658'}
ureq.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/all/USW00014922.dly','USW00014922.dly')
{'USW00022536': 'HI LIHUE WSO AP 1020.1 GSN 91165'}
ureq.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/all/USW00022536.dly','USW00022536.dly')
```

In [ ]:
open('stations.txt','r').readlines()[:10]

In [ ]:
stations = {}

for line in open('stations.txt','r'):
    if 'GSN' in line:
        fields = line.split()
        
        stations[fields[0]] = ' '.join(fields[4:])

In [ ]:
len(stations)

In [ ]:
def findstation(s):
    found = {code: name for code,name in stations.items() if s in name}
    print(found)

In [ ]:
findstation('LIHUE')

In [ ]:
findstation('SAN DIEGO')

In [ ]:
findstation('MINNEAPOLIS')

In [ ]:
findstation('IRKUTSK')

In [ ]:
datastations = ['USW00022536','USW00023188','USW00014922','RSM00030710']

In [ ]:
open('USW00022536.dly','r').readlines()[:10]

In [ ]:
open('readme.txt','r').readlines()[98:121]

In [ ]:
def parsefile(filename):
    return np.genfromtxt(filename,
                         delimiter = dly_delimiter,
                         usecols = dly_usecols,
                         dtype = dly_dtype,
                         names = dly_names)

In [ ]:
dly_delimiter = [11,4,2,4] + [5,1,1,1] * 31
dly_usecols = [1,2,3] + [4*i for i in range(1,32)]
dly_dtype = [np.int32,np.int32,(np.str_,4)] + [np.int32] * 31
dly_names = ['year','month','obs'] + [str(day) for day in range(1,31+1)]

In [ ]:
lihue = parsefile('USW00022536.dly')

In [ ]:
lihue['year']

In [ ]:
def unroll(record):
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate,startdate + np.timedelta64(1,'M'),np.timedelta64(1,'D'))
    
    rows = [(date,record[str(i+1)]/10) for i,date in enumerate(dates)]
    
    return np.array(rows,dtype=[('date','M8[D]'),('value','d')])

In [ ]:
def unrollyear(record):
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate,startdate + np.timedelta64(1,'M'),np.timedelta64(1,'D'))
    
    rows = [(record['year'],date,record[str(i+1)]/10) for i,date in enumerate(dates)]
    
    return np.array(rows,dtype=[('year','uint32'),('date','M8[D]'),('value','d')])

In [ ]:
unroll(lihue[0])

In [ ]:
def getobs(filename,obs):
    return np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])

In [ ]:
getobs('USW00022536.dly','TMIN')

In [ ]:
lihue_tmax = getobs('USW00022536.dly','TMAX')
lihue_tmin = getobs('USW00022536.dly','TMIN')

In [ ]:
pp.plot(lihue_tmax['date'],lihue_tmax['value'])

In [ ]:
def getobs(filename,obs):
    data = np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])
    
    data['value'][data['value'] == -999.9] = np.nan
    
    return data

In [ ]:
lihue_tmax = getobs('USW00022536.dly','TMAX')
lihue_tmin = getobs('USW00022536.dly','TMIN')

In [ ]:
pp.plot(lihue_tmax['date'],lihue_tmax['value'])
pp.plot(lihue_tmin['date'],lihue_tmin['value'])

In [ ]:
np.mean(lihue_tmin['value']), np.mean(lihue_tmax['value'])

In [ ]:
x = np.array([1,3,5,7],'d')
y = np.array([10,5,2,7],'d')

pp.plot(x,y,'o')

pp.axis([0,8,0,12])

In [ ]:
xs = np.linspace(1,7)

In [ ]:
ys = np.interp(xs,x,y)

In [ ]:
pp.plot(xs,ys,'.')
pp.plot(x,y,'o')

pp.axis([0,8,0,12])

In [ ]:
def fillnans(data):
    nan = np.isnan(data['value'])
    
    data['value'][nan] = np.interp(data['date'][nan],data['date'][~nan],data['value'][~nan])

In [ ]:
fillnans(lihue_tmax)

In [ ]:
def fillnans(data):
    dates_float = data['date'].astype(np.float64)
    
    nan = np.isnan(data['value'])
    
    data['value'][nan] = np.interp(dates_float[nan],dates_float[~nan],data['value'][~nan])

In [ ]:
fillnans(lihue_tmax)
fillnans(lihue_tmin)

In [ ]:
np.mean(lihue_tmin['value']), np.mean(lihue_tmax['value'])

In [ ]:
pp.plot(lihue_tmin['date'],lihue_tmin['value'])

In [ ]:
def plot_smoothed(t,win=10):
    smoothed = np.correlate(t['value'],np.ones(win)/win,'same')
    
    pp.plot(t['date'],smoothed)

In [ ]:
pp.plot(lihue_tmin[10000:12000]['date'],lihue_tmin[10000:12000]['value'])

plot_smoothed(lihue_tmin[10000:12000])
plot_smoothed(lihue_tmin[10000:12000],30)

In [ ]:
pp.figure(figsize=(10,6))

for i,code in enumerate(datastations):
    pp.subplot(2,2,i+1)
    
    plot_smoothed(getobs('{}.dly'.format(code),'TMIN'),365)
    plot_smoothed(getobs('{}.dly'.format(code),'TMAX'),365)
    
    pp.title(stations[code])
    pp.axis(xmin=np.datetime64('1952'),xmax=np.datetime64('2012'),ymin=-10,ymax=30)

pp.tight_layout()

In [ ]:
def selectyear(data,year):
    start = np.datetime64('{}'.format(year))
    end = start + np.timedelta64(1,'Y')
    
    return data[(data['date'] >= start) & (data['date'] < end)]['value']

In [ ]:
selectyear(lihue_tmin,1951)

In [ ]:
lihue_tmin_all = np.vstack([selectyear(lihue_tmin,year)[:365] for year in range(1951,2014+1)])

In [ ]:
lihue_tmin_all

In [ ]:
lihue_tmin_all.shape

In [ ]:
lihue_tmin_recordmin = np.min(lihue_tmin_all,axis=0)
lihue_tmin_recordmax = np.max(lihue_tmin_all,axis=0)

In [ ]:
pp.plot(lihue_tmin_recordmax,'.')

In [ ]:
lihue_tmax_all = np.vstack([selectyear(lihue_tmax,year)[:365] for year in range(1951,2014+1)])

In [ ]:
pp.figure(figsize=(12,4))

days = np.arange(1,365+1)

pp.fill_between(days,np.min(lihue_tmin_all,axis=0),np.max(lihue_tmin_all,axis=0),alpha=0.4)
pp.plot(selectyear(lihue_tmin,2009))

pp.fill_between(days,np.min(lihue_tmax_all,axis=0),np.max(lihue_tmax_all,axis=0),alpha=0.4)
pp.plot(selectyear(lihue_tmax,2009))

pp.axis(xmax=365)

In [ ]:
def mediumbyyear(data):
    databyyear = {year: data[data['year'] == year] if year in data['year'].tolist()}
    # rows = [(year, np.mean(fillnans(unroll(row)))) for year, row in data]
    return databyyear

In [ ]:
minneapolis = parsefile('USW00014922.dly')
minneapolis['year'].tolist()

In [ ]:
unrollyear(minneapolis[0])

In [ ]:
minneapolis_tmin = getobs('USW00014922.dly', 'TMIN')
minneapolis_tmin

In [ ]:
pp.plot(minneapolis_tmin['date'], minneapolis_tmin['value'])

In [ ]:
np.mean(minneapolis_tmin['value'])

In [ ]:
fillnans(minneapolis_tmin)

In [ ]:
minneapolis_tmax = getobs('USW00014922.dly', 'TMAX')
minneapolis_tmax

In [ ]:
pp.plot(minneapolis_tmax['date'], minneapolis_tmax['value'])